In [ ]:
# !pip install pandas
import pandas as pd
import requests, json

In [ ]:
# Step 1 - Call the API for available language models
import requests
request = requests.get(f"http://cmdb.vectorspacebio.science/api/get_language_models?vxv_wallet_addr=0076e0b8328203a1beb056caf17a05a78929bd74")
language_models = request.json()
print(language_models)

In [ ]:
# Step 2 - Choose a language model - e.g., Biosciences_0001
# Requires code from step 1

language_model = language_models[0]
print(language_model)

In [ ]:
# Step 3 - Define up to 10 custom features / column labels (up to 100,000 labels upon request)
column_labels = ["NFAT5", "TF", "HIF1A", "TRIM63", "AKT1", "RACK1", "PIK3CA", "NFIA", "EPHB2", "IRF6"]

In [ ]:
# Step 4 - Optional: Submit a context-dependency - e.g., "mitochondria"
context_dependency = ""
# context_dependency = "mitochondria"

In [ ]:
# Step 5 - Call the CMDB API and return a correlation matrix dataset
# This method may take several minutes to complete depending on the request.
# Requires code from step 1-4

# Dependency requirement: pandas
# If using a notebook: %pip install pandas or !pip install pandas
# If using a terminal: pip install pandas

import requests
import pandas as pd

# Specify the types of rows
row_types = "protein_names"

# Retrieve the correlation matrix dataset and hidden relationship matrix
cmd_request = requests.get(f"http://cmdb.vectorspacebio.science/api/get_correlation_matrix?vxv_wallet_addr=0076e0b8328203a1beb056caf17a05a78929bd74&language_model={language_model['name']}&row_types={row_types}&column_labels={'%2C%20'.join(column_labels)}&context_dependency={context_dependency}")

# Retrieve request and convert to dataframes
cm_dataset = pd.DataFrame.from_dict(cmd_request.json())

# Preview the correlation matrix dataset
cm_dataset.head(10)

In [ ]:
# Step 6 - Preprocess the data and output the html heatmap

# This block preprocesses the data, outputs it to the terminal, injects it into a ready-to-go html template and saves the output in json format
# This requires cm_dataset from step 5

data = []

cols = cm_dataset.columns.tolist()
rows = cm_dataset.index.tolist()[:200] # For quick tests limit the number of rows

for col_n, col in enumerate(cols):
    for row_n, row in enumerate(rows):
        data.append([col_n, row_n, round(cm_dataset[col][row], 3)])

# Visualize the data
print("\nColumn labels:\n")
print(cols)
print("\nRows:\n")
print(rows[:10])
print("\nCorrelation scores (sample):\n")
print(data[:10])

# Load the heatmap template
with open("Heatmap/heatmap-template.html", "r") as file_in:
    text = file_in.read()

# Replace the placeholder text with the data above
text = text.replace("vsb_placeholder_column_labels", str(cols))
text = text.replace("vsb_placeholder_rows", str(rows))
text = text.replace("vsb_placeholder_correlation_scores", str(data))

# Adjust the width, height and legend height of the heatmap
# The more rows included in the heatmap, the larger the height must be for visibility
text = text.replace("vsb_placeholder_heatmap_width", "800")
text = text.replace("vsb_placeholder_heatmap_height", "800")
text = text.replace("vsb_placeholder_heatmap_legend_height", "680")

# Save out the updated heatmap file
with open("Heatmap/heatmap.html", "w") as file_out:
    file_out.write(text)

# Save out the JS
json_output = {"cols": cols, "rows": rows, "correlation_scores": data}
with open("Heatmap/output.json", "w") as file_out:
    json.dump(json_output, file_out)

In [342]:
# Step 7 - Expand the network of relationships by submitting the labels to the Protein-Protein Interaction Network (PPIN) API
# This requires the column_labels from step 3

import requests
import json

# Value of 5 is recommended
top_n = 5

max_depth = 3
scoring_threshold = 0.3
ppin_context_dependency = ""
cross_intersection = False

network_results = []
for column_label in column_labels:
    # print(column_label)
    ppin_request = requests.get(f"http://ppin.vectorspacebio.science/api/protein_network?vxv_wallet_addr=d1030ededff0f348992fd4d41ae169dae99bf9c0&query={column_label}&branches={top_n}&max_depth={max_depth}&min_score={scoring_threshold}&context_dependency={ppin_context_dependency}&cross_intersection={cross_intersection}")
    result = json.loads(ppin_request.content.decode())
    # print(result)
    network_results.append(result)

In [348]:
# Step 8 - Preprocess the data and output the html network graph
nodes = []
hidden_relationships = {}
mapping = {}

# Add the root node of the graph
root_node = ""
for col in column_labels:
    nodes.append([root_node, col])

# Add the leaf nodes
for result in network_results:
    for item in result["graph_network"]:
        source_id, source_name, target_id, target_name = item["source"], item["source_name"], item["target"], item["target_name"]
        
        key = f"{source_name}_{target_name}".upper()
        mapping[key] = {"source_id": source_id, "source_name": source_name, "target_id": target_id, "target_name": target_name}
        hidden_relationships[key] = item["known_relationship"]

        if source_name != "" or target_name != "":
            nodes.append([source_name, target_name])
        

# Visualize the nodes and hidden relationship data
print("\nNodes (sample):\n")
print("'' represents the root node\n")
print(nodes[:10])

print("\nHidden relationships (sample):")
print("1 represents a known relationship, 0 represents a hidden relationship\n")
print([{k:hidden_relationships[k]} for k in list(hidden_relationships.keys())[:10]])

print("\nMapping (sample):")
print([{k:mapping[k]} for k in list(mapping.keys())[:10]])

# Load the network graph template, replace the placeholders with the data above, save out network_graph.html
with open("Network/hidden-relationship-network-template.html", "r") as file_in:
    text = file_in.read()

text = text.replace("vsb_placeholder_nodes", str(nodes))
text = text.replace("vsb_placeholder_hidden_relationships", str(hidden_relationships))
text = text.replace("vsb_placeholder_mapping", str(mapping))

with open("Network/hidden-relationship-network.html", "w") as file_out:
    file_out.write(text)

# Save out the JS
json_output = {"nodes": nodes, "hidden_relationships": hidden_relationships}
with open("Network/output.json", "w") as file_out:
    json.dump(json_output, file_out)


Nodes (sample):

'' represents the root node

[['', 'NFAT5'], ['', 'TF'], ['', 'HIF1A'], ['', 'TRIM63'], ['', 'AKT1'], ['', 'RACK1'], ['', 'PIK3CA'], ['', 'NFIA'], ['', 'EPHB2'], ['', 'IRF6']]

Hidden relationships (sample):
1 represents a known relationship, 0 represents a hidden relationship

[{'NFAT5_NFATC2': 0}, {'NFAT5_RREB1': 0}, {'NFAT5_STK39': 0}, {'NFAT5_KLF15': 0}, {'NFAT5_NFATC1': 0}, {'NFATC2_NFATC4': 1}, {'NFATC2_NFAM1': 0}, {'KLF15_MLXIPL': 0}, {'NFATC1_MEF2C': 1}, {'MLXIPL_MLX': 1}]

Mapping (sample):
[{'NFAT5_NFATC2': {'source_id': 'O94916', 'source_name': 'NFAT5', 'target_id': 'Q13469', 'target_name': 'NFATC2'}}, {'NFAT5_RREB1': {'source_id': 'O94916', 'source_name': 'NFAT5', 'target_id': 'Q92766', 'target_name': 'RREB1'}}, {'NFAT5_STK39': {'source_id': 'O94916', 'source_name': 'NFAT5', 'target_id': 'Q9UEW8', 'target_name': 'STK39'}}, {'NFAT5_KLF15': {'source_id': 'O94916', 'source_name': 'NFAT5', 'target_id': 'Q9UIH9', 'target_name': 'KLF15'}}, {'NFAT5_NFATC1': {'sou